##### An example of using foresee from jupyter notebook

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import pandas as pd
import numpy as np
from io import StringIO
import importlib_resources

# import main from foresee.scripts
from foresee.scripts import main


In [3]:
# upload sample time-series dataframe with columns(id, date_stamp, y)

test_data_light_txt = importlib_resources.files('foresee.data').joinpath('test_data_light.csv').read_text()

ts_df = pd.read_csv(StringIO(test_data_light_txt))


ts_df['date_stamp'] = pd.to_datetime(ts_df['date_stamp'])
ts_df.head()


,id,date_stamp,y
0,1,2019-10-01,74.839996
1,1,2019-10-02,73.349998
2,1,2019-10-03,73.870003
3,1,2019-10-04,75.879997
4,1,2019-10-07,75.099998


In [4]:
# sample time-series dataframe with columns(id, date_stamp, y)

ts_df = utils.read_csv('test_data_light.csv')
ts_df['date_stamp'] = pd.to_datetime(ts_df['date_stamp'])
ts_df.head()

NameError: name 'utils' is not defined

In [ ]:
# user defind parameters

# time series values column name: required if input dataframe has more than one column

endog_colname = 'y'

if len(ts_df.columns) > 1 and endog_colname is None:
    raise ValueError('time series column name is required!!!')

# time series frequency
freq = 5

# out of sample forecast length
fcst_length = 10

# available forecasting models
model_list = ['ewm_model', 'fft', 'holt_winters', 'prophet', 'sarimax']

# avilable run types: 'best_model', 'all_best', 'all_models'
run_type = 'all_best'

# if comparing models (run_type in 'best_model' or 'all_best') then holdout length is required

if run_type == 'all_models':
    holdout_length = None
else:
    holdout_length = 20
    
# fit-forecast computations can be done in parallel for each time series. requires dask library!!!
# for sequential processing set fit_execution_method to 'non_parallel'

fit_execution_method = 'parallel'


# since we have two time series in this dataset, time series id column name and date-time column name are required.
gbkey = 'id'
ds_column = 'date_stamp'

In [ ]:
'''
result:  dataframe containing fitted values and future forecasts
fit_results_list:  list of dictionaries containing fitted values, forecasts, and errors (useful for debuging)
'''

result, fit_result_list = main.collect_result(
                                                    ts_df.copy(),
                                                    endog_colname,
                                                    gbkey,
                                                    ds_column, 
                                                    freq, 
                                                    fcst_length, 
                                                    run_type, 
                                                    holdout_length, 
                                                    model_list,
                                                    fit_execution_method,
                                            )

result.head()

In [ ]:
result.tail()

In [ ]:
# m_list = model_list.remove('prophet')
for k, df in result.groupby('ts_id'):
    df.iloc[-50:].plot(y=[x+'_forecast' for x in model_list])


In [ ]:
for item in fit_result_list:
    for model in model_list:
        if item[model+'_err'] is not None:
            print(model + ': ' + item[model+'_err'])